In [1]:
# ===============================================================
# 
# ===============================================================

import numpy as np
import pandas as pd
import time
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score
)

RANDOM_STATE = 42
TEST_SIZE = 0.30

# 1) Carregar e preparar os dados (sem alteração)
cov = fetch_covtype(as_frame=True)
X = cov.data
y_mult = cov.target
y = (y_mult == 1).astype(int)
feature_names = cov.feature_names
class_names = {0: "Outras (0)", 1: "Spruce/Fir (1)"}

print("=== Dataset: Covertype (binário) ===")
print(f"Instâncias totais: {X.shape[0]} | Nº de features: {X.shape[1]}\n")

# 2) Split treino/teste (sem alteração)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)
print(f"Tamanhos -> treino: {X_train.shape[0]} | teste: {X_test.shape[0]}\n")

# 3) Definir os modelos (SVM RBF FOI REMOVIDO/COMENTADO)
modelos = [
    # ("SVM (RBF)", Pipeline([
    #     ("scaler", StandardScaler()),
    #     ("clf", SVC(kernel="rbf", gamma="scale", C=1.0,
    #                 probability=True, random_state=RANDOM_STATE))
    # ])),
    ("SVM (Linear)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LinearSVC(C=1.0, random_state=RANDOM_STATE, max_iter=2000))
    ])),
    ("k-NN (k=5)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=5, n_jobs=-1))
    ])),
    ("MLP (Rede Neural)", Pipeline([
        ("scaler", StandardScaler()),
        ("clf", MLPClassifier(hidden_layer_sizes=(64,),
                              activation="relu",
                              solver="adam",
                              max_iter=1000,
                              random_state=RANDOM_STATE,
                              early_stopping=True))
    ])),
]

# 4) Funções utilitárias (sem alteração)
def avaliar_modelo(nome, modelo, X_train, y_train, X_test, y_test):
    print(f"--- Treinando: {nome} ---")
    start_time = time.time()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    end_time = time.time()
    cm = confusion_matrix(y_test, y_pred)
    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, pos_label=1)
    rec  = recall_score(y_test, y_pred, pos_label=1)
    f1   = f1_score(y_test, y_pred, pos_label=1)
    print(f"=== {nome} ===")
    print(f"Tempo de execução: {end_time - start_time:.2f} segundos")
    print(f"Acurácia: {acc:.3f}")
    print(f"Precisão (positiva=Spruce/Fir): {prec:.3f}")
    print(f"Recall   (positiva=Spruce/Fir): {rec:.3f}")
    print(f"F1       (positiva=Spruce/Fir): {f1:.3f}")
    print("\nMatriz de Confusão (linhas=Real, colunas=Previsto):")
    print("           Prev 0    Prev 1")
    print(f"Real 0  |  {cm[0,0]:>7}   {cm[0,1]:>7}   <- {class_names[0]}")
    print(f"Real 1  |  {cm[1,0]:>7}   {cm[1,1]:>7}   <- {class_names[1]}\n")
    return modelo

def predicoes_individuais(nome, modelo, exemplos):
    print(f"--- Predições Individuais — {nome} ---")
    if hasattr(modelo, "predict_proba"):
        probas = modelo.predict_proba(exemplos)
        preds  = modelo.predict(exemplos)
        for i, (p, pred) in enumerate(zip(probas, preds)):
            idx = exemplos.index[i]
            print(f"Amostra {idx} -> prev={class_names[pred]} | P(Outras)= {p[0]:.3f} | P(Spruce/Fir)= {p[1]:.3f}")
    else:
        preds = modelo.predict(exemplos)
        for i, pred in enumerate(preds):
            idx = exemplos.index[i]
            print(f"Amostra {idx} -> prev={class_names[pred]} (modelo não fornece probabilidade)")
    print()

# 5) Rodar os modelos (sem alteração)
exemplos = X_test.head(3)
for nome, mdl in modelos:
    mfit = avaliar_modelo(nome, mdl, X_train, y_train, X_test, y_test)
    predicoes_individuais(nome, mfit, exemplos)
    print("="*60)

=== Dataset: Covertype (binário) ===
Instâncias totais: 581012 | Nº de features: 54

Tamanhos -> treino: 406708 | teste: 174304

--- Treinando: SVM (Linear) ---
=== SVM (Linear) ===
Tempo de execução: 48.10 segundos
Acurácia: 0.769
Precisão (positiva=Spruce/Fir): 0.693
Recall   (positiva=Spruce/Fir): 0.655
F1       (positiva=Spruce/Fir): 0.674

Matriz de Confusão (linhas=Real, colunas=Previsto):
           Prev 0    Prev 1
Real 0  |    92352     18400   <- Outras (0)
Real 1  |    21927     41625   <- Spruce/Fir (1)

--- Predições Individuais — SVM (Linear) ---
Amostra 219718 -> prev=Outras (0) (modelo não fornece probabilidade)
Amostra 68526 -> prev=Outras (0) (modelo não fornece probabilidade)
Amostra 520898 -> prev=Spruce/Fir (1) (modelo não fornece probabilidade)

--- Treinando: k-NN (k=5) ---
=== k-NN (k=5) ===
Tempo de execução: 519.81 segundos
Acurácia: 0.946
Precisão (positiva=Spruce/Fir): 0.934
Recall   (positiva=Spruce/Fir): 0.918
F1       (positiva=Spruce/Fir): 0.926

Matriz 